In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
class Extractor():
    # input : the url of an openbenchmarking.org webpage
    # output : a dataframe containing the average performances for a configurable system
    
    def __init__(self, url):
        self.url = url
        html_text = requests.get(self.url).text
        self.parser = BeautifulSoup(html_text, 'html.parser')
        # to remove the "Low-Tier"-like lines
        self.banwords = ["Low-Tier", "Median", "Mid-Tier"]
    
    def extract_data(self):
        row_text = self.parser.find_all('div', class_='div_table_row')
        lines = [self.extract_row(rt) for rt in row_text]
        return pd.DataFrame(lines).dropna()
        
    def extract_row(self, row_text):
        content_line = []
        cells = row_text.find_all('div', class_='div_table_cell')
        if cells[0].get_text() not in self.banwords:
            content_line = [c.get_text() for c in cells]
        return content_line

In [3]:
ext = Extractor('https://openbenchmarking.org/test/pts/x264')
df = ext.extract_data()
df

,0,1,2,3
0,AMD EPYC 7763 64-Core,100th,6,236 +/- 3
1,AMD EPYC 75F3 32-Core,100th,8,229 +/- 11
2,AMD EPYC 7713 64-Core,99th,6,225 +/- 2
3,2 x AMD EPYC 75F3 32-Core,99th,13,223 +/- 28
4,2 x AMD EPYC 7713 64-Core,99th,7,222 +/- 3
...,...,...,...,...
176,Intel Pentium Dual E2220,6th,3,5
177,ARMv8 Cortex-A55 4-Core,5th,3,4
178,Intel Pentium Dual T2310,4th,7,3
179,ARMv7 Cortex-A53 4-Core,4th,3,2
